In [ ]:
#r "nuget: Microsoft.SemanticKernel, *-*"
#r "nuget: Microsoft.SemanticKernel.Connectors.AI.HuggingFace, *-*"
#r "nuget: Microsoft.SemanticKernel.Connectors.Memory.Qdrant, *-*"

In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.Memory.Qdrant;
using Microsoft.SemanticKernel.Plugins.Memory;
using Microsoft.SemanticKernel.Connectors.AI.HuggingFace.TextEmbedding;

In [ ]:
string chat_endpoint = "http://localhost:5002/v1/chat/completions";
string embeddings_endpoint = "http://localhost:5002/v1/embeddings";
string qdrant_endpoint = "http://localhost:6333";

In [ ]:

#pragma warning disable SKEXP0020

Kernel kernel = new KernelBuilder()
            .AddHuggingFaceTextGeneration(
                model: "baichuan2",
                endpoint: chat_endpoint)
            .Build();

In [ ]:
#pragma warning disable SKEXP0052
#pragma warning disable CS1061
#pragma warning disable SKEXP0011
#pragma warning disable SKEXP0026

#pragma warning disable SKEXP0020

var qdrantMemoryBuilder = new MemoryBuilder();

var hfembeddings = new HuggingFaceTextEmbeddingGeneration("text2veccn", embeddings_endpoint);

qdrantMemoryBuilder.WithTextEmbeddingGeneration(hfembeddings);
qdrantMemoryBuilder.WithQdrantMemoryStore(qdrant_endpoint, 1024);

var builder = qdrantMemoryBuilder.Build();

In [ ]:
string MemoryCollectionName = "demovb";

await builder.SaveInformationAsync(MemoryCollectionName, id: "id1", text: "卢建晖是微软高级云技术布道师");
await builder.SaveInformationAsync(MemoryCollectionName, id: "id2", text: "卢建晖是微软最有价值专家");
await builder.SaveInformationAsync(MemoryCollectionName, id: "id3", text: "卢建晖是微软黑客松 AI 教练");
await builder.SaveInformationAsync(MemoryCollectionName, id: "id4", text: "蜡笔小新出自日本漫画");


In [ ]:
var searchResults =  builder.SearchAsync(MemoryCollectionName, "介绍一下卢建晖", limit: 3, minRelevanceScore: 0.6);

In [ ]:
await foreach (var item in searchResults)
{
    Console.WriteLine(item.Metadata.Text + " : " + item.Relevance);
}

In [ ]:
var questionAnswerFunction = kernel.CreateFunctionFromPrompt("请回答问题，用户输入 {{$input}} ");

In [ ]:
var result = await kernel.InvokeAsync(questionAnswerFunction, new("能帮我把这个题目改得更吸引人吗?:<<提升教师数字素养:激发教师信息化教学创新力量实操工作坊>>"));


In [ ]:
result.GetValue<string>()